In [134]:
import pandas as pd
import csv
import re
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, GRU, Dense, Bidirectional,Dropout, SimpleRNN
from sklearn.metrics import accuracy_score, precision_score, recall_score,f1_score, classification_report
import nltk
from nltk.corpus import stopwords
import spacy

In [42]:
pip install spacy

  Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
  Attempting uninstall: colorama
    Found existing installation: colorama 0.4.4
    Uninstalling colorama-0.4.4:
      Successfully uninstalled colorama-0.4.4



In [110]:
data = pd.read_table(r'C:\Users\Mannahil Miftah\Downloads\urdu-sentiment-corpus-v1.tsv')
data

,Tweet,Class
0,میں نے ایٹم بم بنایا ھے ۔۔۔۔او بھائی ایٹم بمب ...,P
1,چندے سے انقلاب اور عمران خان وزیر اعظم نہیں بن...,N
2,ٹویٹر کا خیال کیسے آیا ؟,O
3,"سرچ انجن گوگل کے نائب صدر نے فضا میں ، 130,000...",P
4,ابھی تک اسکی لہریں کبھی کبھی آ جاتی ہیں یار :أْ,P
...,...,...
995,اُس آدمی نے اِس سالار کو کافی معقول ٹپ دی ہے ۔,P
996,چچا غالب کی روح سے معذرت کے ساتھہم نے مانا کہ ...,P
997,واہ جناب واہ! اچھی رہی۔ جناب خود کو فرشتہ سمجو...,P
998,اسلام آباد :پی اے ٹی کا دھرنا ختم، صفائی کے کا...,P


In [111]:
# saving the data into lists; Text and Labels

d = r"C:\Users\Mannahil Miftah\Downloads\urdu-sentiment-corpus-v1.tsv"

Text = []
Label = []

with open(d, 'r', encoding = 'utf-8') as file:
    data_csv = csv.reader(file, delimiter = '\t')
    next(data_csv)

    for row in data_csv:
        # any row which doesnot contain either text, label or both will be skipped
        if len(row) >= 2:
            Text.append(row[0].strip())
            Label.append(row[1])

In [112]:
# removing unnecessary charactes from the text

for i in range(len(Text)):
    Text[i] = re.sub(r'[^\w\s]', '', Text[i])

In [113]:
# as we have to do binary classification (P and N), so removing the text with label O

text = []
label = []
for i in range(len(Text)):
    if Label[i] == 'P' or Label[i] == 'N':
        text.append(Text[i])
        label.append(Label[i])

In [114]:
# printing the data after cleaning

for i in range(0,5):
    print("Text: ",text[i])
    print("Label: ",label[i])

Text:  میں نے ایٹم بم بنایا ھے او بھائی ایٹم بمب کوٹ لکھپت والی اتفاق فیکٹری میں نہیں بنتاایٹم بم کہوٹہ کی ایٹمی
Label:  P
Text:  چندے سے انقلاب اور عمران خان وزیر اعظم نہیں بن سکتے
Label:  N
Text:  سرچ انجن گوگل کے نائب صدر نے فضا میں  130000 فٹ کی بلندی پر چھلانگ لگا کر عالمی ریکارڈ قائم کرلیا چھلانگ کی
Label:  P
Text:  ابھی تک اسکی لہریں کبھی کبھی آ جاتی ہیں یار أ
Label:  P
Text:  گندی زبان اور گٹر جیسے دماغ والے جاهل جیالے هو تم جیالا هو اور جاهل نه هو یه ممکن نهیں 
Label:  N


In [115]:
# splitting the data

X_train, X_test, y_train, y_test = train_test_split(text, label, test_size = 0.25, random_state = 42)

In [116]:
# Encoding labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [117]:
# tokenizing

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [167]:
# padding the sequences to ensure that shorter sequences will have same length as the longest sequence or max length

max_length = 200
X_train = pad_sequences(X_train, maxlen = max_length, padding = 'post')
X_test = pad_sequences(X_test, maxlen = max_length, padding = 'post')

In [168]:
number_of_layers = [2, 3]
dropout_rates = [0.3, 0.7]
rnn_result = []
gru_result = []
lstm_result = []
bilstm_result = []

##### RNN MODEL

In [189]:
# Defining RNN model

def rnn(layers, dropout_rate):
    model_rnn = Sequential()
    model_rnn.add(Embedding(input_dim = len(tokenizer.word_index) + 1, output_dim = 100, input_length = max_length))

    for _ in range(layers - 1):
        model_rnn.add(SimpleRNN(units = 64, return_sequences = True))
        model_rnn.add(Dropout(dropout_rate))

    model_rnn.add(SimpleRNN(units = 64))
    model_rnn.add(Dropout(dropout_rate))
    model_rnn.add(Dense(units = 1, activation = 'sigmoid'))

    # Compiling and training the RNN model
    model_rnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    model_rnn.fit(X_train, y_train, epochs = 10, batch_size = 20, validation_data = (X_test, y_test))
    return model_rnn

In [190]:
for n in number_of_layers:
    for d in dropout_rates:
        model = rnn(n, d)
        pred_rnn = np.round(model.predict(X_test))
        accuracy_rnn = accuracy_score(y_test, pred_rnn)
        precision_rnn = precision_score(y_test, pred_rnn, zero_division = 1)
        recall_rnn = recall_score(y_test, pred_rnn)
        f1_rnn = f1_score(y_test, pred_rnn)      
        rnn_result.append({
            'Number of layers':n,
            'Dropout rate':d,
            'Accuracy':accuracy_rnn,
            'Precision':precision_rnn,
            'Recall':recall_rnn,
            'F1 score':f1_rnn
        })

Epoch 1/10
37/37 [==============================] - 5s 85ms/step - loss: 0.7339 - accuracy: 0.4905 - val_loss: 0.7234 - val_accuracy: 0.4735
Epoch 2/10
37/37 [==============================] - 3s 79ms/step - loss: 0.7294 - accuracy: 0.5054 - val_loss: 0.7368 - val_accuracy: 0.4816
Epoch 3/10
37/37 [==============================] - 3s 79ms/step - loss: 0.7242 - accuracy: 0.5191 - val_loss: 0.7375 - val_accuracy: 0.5061
Epoch 4/10
37/37 [==============================] - 3s 75ms/step - loss: 0.7492 - accuracy: 0.4918 - val_loss: 0.6961 - val_accuracy: 0.4857
Epoch 5/10
37/37 [==============================] - 3s 88ms/step - loss: 0.7359 - accuracy: 0.4782 - val_loss: 0.7129 - val_accuracy: 0.4939
Epoch 6/10
37/37 [==============================] - 3s 80ms/step - loss: 0.7311 - accuracy: 0.5041 - val_loss: 0.6991 - val_accuracy: 0.4694
Epoch 7/10
37/37 [==============================] - 3s 79ms/step - loss: 0.7332 - accuracy: 0.4877 - val_loss: 0.6941 - val_accuracy: 0.5306
Epoch 8/10
37

In [191]:
# printing results

rnn_result

[{'Number of layers': 2,
  'Dropout rate': 0.3,
  'Accuracy': 0.5142857142857142,
  'Precision': 0.4830508474576271,
  'Recall': 0.4956521739130435,
  'F1 score': 0.4892703862660944},
 {'Number of layers': 2,
  'Dropout rate': 0.7,
  'Accuracy': 0.5142857142857142,
  'Precision': 0.4830508474576271,
  'Recall': 0.4956521739130435,
  'F1 score': 0.4892703862660944},
 {'Number of layers': 3,
  'Dropout rate': 0.3,
  'Accuracy': 0.5142857142857142,
  'Precision': 0.4830508474576271,
  'Recall': 0.4956521739130435,
  'F1 score': 0.4892703862660944},
 {'Number of layers': 3,
  'Dropout rate': 0.7,
  'Accuracy': 0.5142857142857142,
  'Precision': 0.4830508474576271,
  'Recall': 0.4956521739130435,
  'F1 score': 0.4892703862660944},
 {'Number of layers': 2,
  'Dropout rate': 0.3,
  'Accuracy': 0.4816326530612245,
  'Precision': 0.4423076923076923,
  'Recall': 0.4,
  'F1 score': 0.4200913242009133},
 {'Number of layers': 2,
  'Dropout rate': 0.7,
  'Accuracy': 0.47346938775510206,
  'Precision

In [192]:
print(classification_report(y_test, pred_rnn))

              precision    recall  f1-score   support

           0       0.53      1.00      0.69       130
           1       0.00      0.00      0.00       115

    accuracy                           0.53       245
   macro avg       0.27      0.50      0.35       245
weighted avg       0.28      0.53      0.37       245



c:\Users\Mannahil Miftah\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Mannahil Miftah\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Mannahil Miftah\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### GRU MODEL

In [211]:
# Defining GRU model

def gru(layers, dropout_rate):
    model_gru = Sequential()
    model_gru.add(Embedding(input_dim = len(tokenizer.word_index) + 1, output_dim = 100, input_length = max_length))

    for _ in range(layers - 1):
        model_gru.add(GRU(units = 64, return_sequences = True))
        model_gru.add(Dropout(dropout_rate))

    model_gru.add(GRU(units = 64))
    model_gru.add(Dropout(dropout_rate))
    model_gru.add(Dense(units = 1, activation = 'sigmoid'))

    # Compiling and training the GRU model
    model_gru.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    model_gru.fit(X_train, y_train, epochs = 10, batch_size = 20, validation_data = (X_test, y_test))
    return model_gru

In [212]:
for n in number_of_layers:
    for d in dropout_rates:
        model = gru(n, d)
        pred_gru = np.round(model.predict(X_test))
        accuracy_gru = accuracy_score(y_test, pred_gru)
        precision_gru = precision_score(y_test, pred_gru, zero_division = 1)
        recall_gru = recall_score(y_test, pred_gru)
        f1_gru = f1_score(y_test, pred_gru)      
        gru_result.append({
            'Number of layers':n,
            'Dropout rate':d,
            'Accuracy':accuracy_gru,
            'Precision':precision_gru,
            'Recall':recall_gru,
            'F1 score':f1_gru
        })

Epoch 1/10
37/37 [==============================] - 18s 265ms/step - loss: 0.6945 - accuracy: 0.5054 - val_loss: 0.6961 - val_accuracy: 0.4694
Epoch 2/10
37/37 [==============================] - 9s 236ms/step - loss: 0.6958 - accuracy: 0.5068 - val_loss: 0.6923 - val_accuracy: 0.5306
Epoch 3/10
37/37 [==============================] - 7s 186ms/step - loss: 0.6945 - accuracy: 0.4946 - val_loss: 0.6923 - val_accuracy: 0.5306
Epoch 4/10
37/37 [==============================] - 7s 200ms/step - loss: 0.6942 - accuracy: 0.4700 - val_loss: 0.6923 - val_accuracy: 0.5306
Epoch 5/10
37/37 [==============================] - 7s 192ms/step - loss: 0.6938 - accuracy: 0.4946 - val_loss: 0.6930 - val_accuracy: 0.5306
Epoch 6/10
37/37 [==============================] - 17s 471ms/step - loss: 0.6944 - accuracy: 0.4823 - val_loss: 0.6921 - val_accuracy: 0.5306
Epoch 7/10
37/37 [==============================] - 17s 467ms/step - loss: 0.6966 - accuracy: 0.4700 - val_loss: 0.6917 - val_accuracy: 0.5306
Epo

In [213]:
# printing results

gru_result

[{'Number of layers': 2,
  'Dropout rate': 0.3,
  'Accuracy': 0.5061224489795918,
  'Precision': 0.47368421052631576,
  'Recall': 0.46956521739130436,
  'F1 score': 0.47161572052401746},
 {'Number of layers': 2,
  'Dropout rate': 0.7,
  'Accuracy': 0.5061224489795918,
  'Precision': 0.47368421052631576,
  'Recall': 0.46956521739130436,
  'F1 score': 0.47161572052401746},
 {'Number of layers': 3,
  'Dropout rate': 0.3,
  'Accuracy': 0.5061224489795918,
  'Precision': 0.47368421052631576,
  'Recall': 0.46956521739130436,
  'F1 score': 0.47161572052401746},
 {'Number of layers': 3,
  'Dropout rate': 0.7,
  'Accuracy': 0.5061224489795918,
  'Precision': 0.47368421052631576,
  'Recall': 0.46956521739130436,
  'F1 score': 0.47161572052401746},
 {'Number of layers': 2,
  'Dropout rate': 0.3,
  'Accuracy': 0.5142857142857142,
  'Precision': 0.4830508474576271,
  'Recall': 0.4956521739130435,
  'F1 score': 0.4892703862660944},
 {'Number of layers': 2,
  'Dropout rate': 0.7,
  'Accuracy': 0.5142

In [214]:
print(classification_report(y_test, pred_gru))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       130
           1       0.47      1.00      0.64       115

    accuracy                           0.47       245
   macro avg       0.23      0.50      0.32       245
weighted avg       0.22      0.47      0.30       245



c:\Users\Mannahil Miftah\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Mannahil Miftah\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Mannahil Miftah\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### LSTM MODEL

In [196]:
# defining LSTM model

def lstm(layers, dropout_rate):
    model_lstm = Sequential()
    model_lstm.add(Embedding(input_dim = len(tokenizer.word_index) + 1, output_dim = 100, input_length = max_length))

    for _ in range(layers - 1):
        model_lstm.add(LSTM(units = 64, return_sequences = True))
        model_lstm.add(Dropout(dropout_rate))

    model_lstm.add(LSTM(units = 64))
    model_lstm.add(Dropout(dropout_rate))
    model_lstm.add(Dense(units = 1, activation = 'sigmoid'))

    # compiling and training the LSTM model
    model_lstm.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    model_lstm.fit(X_train, y_train, epochs = 10, batch_size = 20, validation_data = (X_test, y_test), verbose = 0)
    return model_lstm

In [197]:
for n in number_of_layers:
    for d in dropout_rates:
        model = lstm(n, d)
        pred_lstm = np.round(model.predict(X_test))
        accuracy_lstm = accuracy_score(y_test, pred_lstm)
        precision_lstm = precision_score(y_test, pred_lstm, zero_division = 1)
        recall_lstm = recall_score(y_test, pred_lstm)
        f1_lstm = f1_score(y_test, pred_lstm)      
        lstm_result.append({
            'Number of layers':n,
            'Dropout rate':d,
            'Accuracy':accuracy_lstm,
            'Precision':precision_lstm,
            'Recall':recall_lstm,
            'F1 score':f1_lstm
        })

8/8 [==============================] - 3s 173ms/step


In [199]:
# printing the results

lstm_result

[{'Number of layers': 2,
  'Dropout rate': 0.3,
  'Accuracy': 0.5306122448979592,
  'Precision': 1.0,
  'Recall': 0.0,
  'F1 score': 0.0},
 {'Number of layers': 2,
  'Dropout rate': 0.7,
  'Accuracy': 0.46938775510204084,
  'Precision': 0.46938775510204084,
  'Recall': 1.0,
  'F1 score': 0.6388888888888888},
 {'Number of layers': 3,
  'Dropout rate': 0.3,
  'Accuracy': 0.46938775510204084,
  'Precision': 0.46938775510204084,
  'Recall': 1.0,
  'F1 score': 0.6388888888888888},
 {'Number of layers': 3,
  'Dropout rate': 0.7,
  'Accuracy': 0.5306122448979592,
  'Precision': 1.0,
  'Recall': 0.0,
  'F1 score': 0.0}]

In [200]:
print(classification_report(y_test, pred_lstm))

              precision    recall  f1-score   support

           0       0.53      1.00      0.69       130
           1       0.00      0.00      0.00       115

    accuracy                           0.53       245
   macro avg       0.27      0.50      0.35       245
weighted avg       0.28      0.53      0.37       245



c:\Users\Mannahil Miftah\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Mannahil Miftah\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Mannahil Miftah\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


##### BILSTM MODEL

In [205]:
# defining BiLSTM model

def bilstm(layers, dropout_rate):
    model_bilstm = Sequential()
    model_bilstm.add(Embedding(input_dim = len(tokenizer.word_index) + 1, output_dim = 100, input_length = max_length))

    for _ in range(layers - 1):
        model_bilstm.add(Bidirectional(LSTM(units = 64, return_sequences = True)))
        model_bilstm.add(Dropout(dropout_rate))

    model_bilstm.add(Bidirectional(LSTM(units = 64)))
    model_bilstm.add(Dropout(dropout_rate))
    model_bilstm.add(Dense(units = 1, activation = 'sigmoid'))

    # compiling and training the BiLSTM model
    model_bilstm.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    model_bilstm.fit(X_train, y_train, epochs = 10, batch_size = 20, validation_data = (X_test, y_test), verbose = 0)
    return model_bilstm

In [206]:
for n in number_of_layers:
    for d in dropout_rates:
        model = bilstm(n, d)
        pred_bilstm = np.round(model.predict(X_test))
        accuracy_bilstm = accuracy_score(y_test, pred_bilstm)
        precision_bilstm = precision_score(y_test, pred_bilstm, zero_division = 1)
        recall_bilstm = recall_score(y_test, pred_bilstm)
        f1_bilstm = f1_score(y_test, pred_bilstm)
        bilstm_result.append({
            'Number of layers':n,
            'Dropout rate':d,
            'Accuracy':accuracy_bilstm,
            'Precision':precision_bilstm,
            'Recall':recall_bilstm,
            'F1 score':f1_bilstm
        })

8/8 [==============================] - 7s 528ms/step


In [207]:
# printing the results

bilstm_result

[{'Number of layers': 2,
  'Dropout rate': 0.3,
  'Accuracy': 0.5836734693877551,
  'Precision': 0.5355191256830601,
  'Recall': 0.8521739130434782,
  'F1 score': 0.6577181208053691},
 {'Number of layers': 2,
  'Dropout rate': 0.7,
  'Accuracy': 0.6571428571428571,
  'Precision': 0.624,
  'Recall': 0.6782608695652174,
  'F1 score': 0.65},
 {'Number of layers': 3,
  'Dropout rate': 0.3,
  'Accuracy': 0.6163265306122448,
  'Precision': 0.5755395683453237,
  'Recall': 0.6956521739130435,
  'F1 score': 0.6299212598425197},
 {'Number of layers': 3,
  'Dropout rate': 0.7,
  'Accuracy': 0.6204081632653061,
  'Precision': 0.5859375,
  'Recall': 0.6521739130434783,
  'F1 score': 0.6172839506172839}]

In [208]:
print(classification_report(y_test, pred_bilstm))

              precision    recall  f1-score   support

           0       0.66      0.59      0.62       130
           1       0.59      0.65      0.62       115

    accuracy                           0.62       245
   macro avg       0.62      0.62      0.62       245
weighted avg       0.62      0.62      0.62       245

